# Wind residuals (hourly mean and altitude average 223-283 km).
## It has the option of selecting a SLT range, latitude range and longitude range.
## This script uses as quiet time two days before the storms but not the day immediately before the storm.
## This script prints the min and max SLT, longitude and latitude.

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

# Directory for saving plots
save_directory = "/Users/gildagonzalez/Documents/Datos_Plots/SEA/SEA_2020-2022/MIGHTI_Winds/Residuals/SLT_Long"
if not os.path.exists(save_directory):
    os.makedirs(save_directory)
print(f"Directory '{save_directory}' is ready for saving plots.")

def load_combined_data(file_path):
    print(f"Loading combined data from {file_path}...")
    try:
        df = pd.read_csv(file_path, sep='\s+', dtype=str, skiprows=1,
                         names=['YEAR', 'DOY', 'HOUR', 'MINUTE', 'Bz', 'Ey', 'SYM-H'])
        
        if df.iloc[0]['YEAR'].startswith('YEAR'):
            df = df.iloc[1:]
        
        df.replace({'9999.99': np.nan, '999.99': np.nan}, inplace=True)
        df[['Bz', 'Ey', 'SYM-H']] = df[['Bz', 'Ey', 'SYM-H']].apply(pd.to_numeric, errors='coerce')
        
        df['DOY'] = df['DOY'].apply(lambda x: x.zfill(3))
        df['HOUR'] = df['HOUR'].apply(lambda x: x.zfill(2))
        df['MINUTE'] = df['MINUTE'].apply(lambda x: x.zfill(2))
        
        df['datetime_str'] = df['YEAR'] + df['DOY'] + df['HOUR'] + df['MINUTE']
        
        df['datetime'] = pd.to_datetime(df['datetime_str'], format='%Y%j%H%M')
        
        df.set_index('datetime', inplace=True)
        df.drop(['YEAR', 'DOY', 'HOUR', 'MINUTE', 'datetime_str'], axis=1, inplace=True)

        print("Data loaded successfully.")
        return df
    except Exception as e:
        print(f"Error loading combined data: {e}")
        return None

def identify_storm_events(sym_h_series, threshold=-50):
    print("Identified storm events with their minimum SYM-H values:")
    count = 0
    for datetime, value in sym_h_series.items():
        if value <= threshold:
            print(f"{datetime}: SYM-H = {value}")
            count += 1
    print(f"Total number of detected storm events: {count}")
    storm_events_mask = sym_h_series <= threshold
    storm_events = sym_h_series[storm_events_mask]
    return storm_events

def find_min_sym_h_within_events(sym_h_data, storm_events, window_days=1):
    print("Finding minimum SYM-H times within storm events...")
    min_sym_h_times = []

    for datetime, value in storm_events.items():
        window_start = datetime - pd.Timedelta(days=window_days)
        window_end = datetime + pd.Timedelta(days=window_days)
        
        # Check if the window range exists in the index and has non-NaN data
        if sym_h_data.loc[window_start:window_end].notnull().any().all():
            window_data = sym_h_data.loc[window_start:window_end]
            min_time = window_data.idxmin()
            min_sym_h_times.append(min_time)
        else:
            print(f"Skipping analysis for window around {datetime} due to insufficient data.")

    print("Minimum SYM-H times identified for all applicable storm events.")
    return min_sym_h_times

def load_wind_data(file_path, lat_range=None, lon_range=None):
    print(f"Loading wind data from {file_path}...")
    try:
        # Include 'ICON_L22_Latitude' and 'ICON_L22_Longitude' in usecols if needed
        cols = ['UTC Time', 'Meridional Wind (m/s)', 'Zonal Wind (m/s)', 'Solar Local Time']
        if lat_range or lon_range:  # Add latitude and longitude columns if any range is specified
            cols.append('ICON_L22_Latitude')
            cols.append('ICON_L22_Longitude')
        df = pd.read_csv(file_path, usecols=cols)
        
        # Parse 'UTC Time' to datetime, handling milliseconds
        df['UTC Time'] = pd.to_datetime(df['UTC Time'], errors='coerce', infer_datetime_format=True)
        df.dropna(subset=['UTC Time'], inplace=True)
        df.set_index('UTC Time', inplace=True)
        df['Solar Local Time'] = df['Solar Local Time'].astype(float)
        
        if lat_range:  # Apply latitude filtering if specified
            df['ICON_L22_Latitude'] = df['ICON_L22_Latitude'].astype(float)
            df = df[(df['ICON_L22_Latitude'] >= lat_range[0]) & (df['ICON_L22_Latitude'] <= lat_range[1])]
        if lon_range:  # Apply longitude filtering if specified
            df['ICON_L22_Longitude'] = df['ICON_L22_Longitude'].astype(float)
            df = df[(df['ICON_L22_Longitude'] >= lon_range[0]) & (df['ICON_L22_Longitude'] <= lon_range[1])]
        
        print("Wind data loaded successfully.")
        print(f"Columns in wind data: {df.columns}")
        return df
    except Exception as e:
        print(f"Error loading wind data: {e}")
        return None

def calculate_hours_since_min_sym_h(data, min_sym_h_times):
    sorted_min_sym_h_times = sorted(min_sym_h_times)
    data['Hours Since Min SYM-H'] = np.nan  # Ensure this column is in your DataFrame
    
    for i, min_sym_h_time in enumerate(sorted_min_sym_h_times):
        next_min_time = sorted_min_sym_h_times[i + 1] if i + 1 < len(sorted_min_sym_h_times) else data.index.max()
        mask = (data.index >= min_sym_h_time) & (data.index < next_min_time)
        
        if mask.any():
            data.loc[mask, 'Hours Since Min SYM-H'] = (data.loc[mask].index - min_sym_h_time) / pd.Timedelta(hours=1)

    return data

def sanitize_filename(filename):
    """Sanitize the parameter name to be used in filenames."""
    return filename.replace("/", "_per_").replace(" ", "_")

def perform_sea_and_plot_individual_figures(combined_data, min_sym_h_times, wind_data, save_directory, window_days=3, lat_range=None, lon_range=None, y_labels=None, x_label='Days', point_size=30, slt_start=18, slt_end=23, xlabel_size=16, ylabel_size=16, title_size=16, legend_size=12, xtick_label_size=20, ytick_label_size=20, quiet_days=1):
    def calculate_sea_results(data, param, min_sym_h_times, window_days, slt_start, slt_end, lat_range, lon_range, quiet_days):
        sea_results = pd.DataFrame()
        filtered_slt_values = []
        filtered_latitude_values = []
        filtered_longitude_values = []

        print(f"SLT Filter Range: {slt_start} to {slt_end}")
        print(f"Available columns in data: {data.columns}")

        for min_time in min_sym_h_times:
            min_time = pd.to_datetime(min_time)

            quiet_period_start = min_time - pd.Timedelta(days=quiet_days + 1)
            quiet_period_end = min_time - pd.Timedelta(days=1)

            quiet_period_data = data.loc[quiet_period_start:quiet_period_end].copy()
            if lat_range and 'ICON_L22_Latitude' in quiet_period_data.columns:
                quiet_period_data = quiet_period_data[(quiet_period_data['ICON_L22_Latitude'] >= lat_range[0]) & (quiet_period_data['ICON_L22_Latitude'] <= lat_range[1])]
            if lon_range and 'ICON_L22_Longitude' in quiet_period_data.columns:
                quiet_period_data = quiet_period_data[(quiet_period_data['ICON_L22_Longitude'] >= lon_range[0]) & (quiet_period_data['ICON_L22_Longitude'] <= lon_range[1])]

            if not quiet_period_data.empty:
                quiet_mean = quiet_period_data[param].mean()
            else:
                quiet_mean = np.nan

            plot_window_start = min_time - pd.Timedelta(days=window_days)
            plot_window_end = min_time + pd.Timedelta(days=window_days)

            plot_window_data = data.loc[plot_window_start:plot_window_end].copy()
            if lat_range and 'ICON_L22_Latitude' in plot_window_data.columns:
                plot_window_data = plot_window_data[(plot_window_data['ICON_L22_Latitude'] >= lat_range[0]) & (plot_window_data['ICON_L22_Latitude'] <= lat_range[1])]
                filtered_latitude_values.extend(plot_window_data['ICON_L22_Latitude'].dropna().values)
            if lon_range and 'ICON_L22_Longitude' in plot_window_data.columns:
                plot_window_data = plot_window_data[(plot_window_data['ICON_L22_Longitude'] >= lon_range[0]) & (plot_window_data['ICON_L22_Longitude'] <= lon_range[1])]
                filtered_longitude_values.extend(plot_window_data['ICON_L22_Longitude'].dropna().values)

            # Apply SLT filtering if 'Solar Local Time' column exists
            if 'Solar Local Time' in plot_window_data.columns:
                pre_filter_count = len(plot_window_data)
                plot_window_data = plot_window_data[(plot_window_data['Solar Local Time'] >= slt_start) & (plot_window_data['Solar Local Time'] <= slt_end)]
                post_filter_count = len(plot_window_data)
                filtered_slt_values.extend(plot_window_data['Solar Local Time'].dropna().values)
                print(f"Filtered SLT data from {pre_filter_count} to {post_filter_count} entries")
                if post_filter_count > 0:
                    print(f"After filtering: Min SLT = {plot_window_data['Solar Local Time'].min()}, Max SLT = {plot_window_data['Solar Local Time'].max()}")

            plot_window_data['residual'] = plot_window_data[param] - quiet_mean
            plot_window_data['Hours Since Min SYM-H'] = ((plot_window_data.index - min_time) / pd.Timedelta(hours=1)).round()
            aggregated_data = plot_window_data.groupby('Hours Since Min SYM-H')['residual'].agg(['mean', 'std']).reset_index()
            sea_results = pd.concat([sea_results, aggregated_data])

        if not sea_results.empty:
            final_aggregated_data = sea_results.groupby('Hours Since Min SYM-H').agg({'mean': 'mean', 'std': 'mean'}).reset_index()

            min_slt = min(filtered_slt_values) if filtered_slt_values else None
            max_slt = max(filtered_slt_values) if filtered_slt_values else None
            min_lat = min(filtered_latitude_values) if filtered_latitude_values else None
            max_lat = max(filtered_latitude_values) if filtered_latitude_values else None
            min_lon = min(filtered_longitude_values) if filtered_longitude_values else None
            max_lon = max(filtered_longitude_values) if filtered_longitude_values else None

            if min_slt is not None and max_slt is not None:
                print(f"Min SLT in plots: {min_slt} hours, Max SLT in plots: {max_slt} hours")
            if min_lat is not None and max_lat is not None:
                print(f"Latitude range in plots: {min_lat}° to {max_lat}°")
            if min_lon is not None and max_lon is not None:
                print(f"Longitude range in plots: {min_lon}° to {max_lon}°")

            return final_aggregated_data, min_slt, max_slt, min_lat, max_lat, min_lon, max_lon
        else:
            return pd.DataFrame(columns=['Hours Since Min SYM-H', 'mean', 'std']), None, None, None, None

    y_labels_map = {
        'SYM-H': 'Mean SYM-H (nT)',
        'Bz': 'Mean Bz (nT)',
        'Ey': 'Mean Ey (mV/m)',
        'Meridional Wind (m/s)': 'Meridional Wind Residuals (m/s)',
        'Zonal Wind (m/s)': 'Zonal Wind Residuals (m/s)'
    }

    parameters = ['SYM-H', 'Bz', 'Ey', 'Meridional Wind (m/s)', 'Zonal Wind (m/s)']

    for i, param in enumerate(parameters):
        fig, ax = plt.subplots(figsize=(10, 5))
        data_source = wind_data if param in ['Meridional Wind (m/s)', 'Zonal Wind (m/s)'] else combined_data
        sea_results, min_slt, max_slt, min_lat, max_lat, min_lon, max_lon = calculate_sea_results(data_source, param, min_sym_h_times, window_days, slt_start, slt_end, lat_range, lon_range, quiet_days)

        y_label = y_labels_map[param]

        if param in ['Meridional Wind (m/s)', 'Zonal Wind (m/s)']:
            ax.errorbar(sea_results['Hours Since Min SYM-H'] / 24, sea_results['mean'],
                        yerr=sea_results['std'], fmt='o', color='black', ecolor='gray',
                        elinewidth=1, capsize=3, alpha=0.6, label='Residuals ± Std Dev')

            slt_range_label = f"SLT Range: {min_slt:.2f} - {max_slt:.2f} hours" if min_slt is not None and max_slt is not None else "SLT Range: N/A"
            lat_range_label = f"Latitude Range: {min_lat:.2f}° - {max_lat:.2f}°" if min_lat is not None and max_lat is not None else "Latitude Range: N/A"
            lon_range_label = f"Longitude Range: {min_lon:.2f}° - {max_lon:.2f}°" if min_lon is not None and max_lon is not None else "Longitude Range: N/A"
            ax.plot([], [], ' ', label=slt_range_label)
            ax.plot([], [], ' ', label=lat_range_label)
            ax.plot([], [], ' ', label=lon_range_label)
        else:
            ax.plot(sea_results['Hours Since Min SYM-H'] / 24, sea_results['mean'], label='Mean', color='black', linewidth=2)
            ax.fill_between(sea_results['Hours Since Min SYM-H'] / 24, sea_results['mean'] - sea_results['std'],
                            sea_results['mean'] + sea_results['std'], color='lightgray', alpha=0.5, label='Std Dev')

        ax.set_ylabel(y_label, fontsize=ylabel_size)
        ax.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero Epoch')
        ax.tick_params(axis='x', labelsize=xtick_label_size)
        ax.tick_params(axis='y', labelsize=ytick_label_size)
        ax.set_xlabel(x_label, fontsize=xlabel_size)
        ax.legend(fontsize=legend_size)
        ax.grid(True)
        plt.tight_layout()

        sanitized_param = sanitize_filename(param)
        plot_filename = os.path.join(save_directory, f"SEA_Result_{sanitized_param}.png")
        plt.savefig(plot_filename)
        plt.close(fig)

def run_analysis(data_file, wind_data_file, save_directory, lat_range=(-90, 90), lon_range=(0, 360)):  
    print("Starting analysis...")
    combined_data = load_combined_data(data_file)
    wind_data = load_wind_data(wind_data_file, lat_range=lat_range, lon_range=lon_range)
    
    if combined_data is not None and wind_data is not None:
        sym_h_hourly = combined_data['SYM-H'].resample('h').mean()  
        storm_events = identify_storm_events(sym_h_hourly, -50)
        min_sym_h_times = find_min_sym_h_within_events(sym_h_hourly, storm_events)
        perform_sea_and_plot_individual_figures(combined_data, min_sym_h_times, wind_data, save_directory, window_days=3, lat_range=lat_range, lon_range=lon_range, quiet_days=1)
        print("Analysis complete.")
    else:
        print("Failed to load necessary data. Analysis cannot proceed.")

# Example file paths, adjust as needed
data_file_path = '/Users/gildagonzalez/Documents/Datos_Plots/Storms/Datos geomag/2020-2022.dat'
wind_data_file_path = '/Users/gildagonzalez/Documents/Datos_Plots/SEA/MIGHTI/With_Long/Wind_Data_2020_2022_Long.dat'

run_analysis(data_file_path, wind_data_file_path, save_directory, lat_range=(-90, 90), lon_range=(0, 360))



Directory '/Users/gildagonzalez/Documents/Datos_Plots/SEA/SEA_2020-2022/MIGHTI_Winds/Residuals/SLT_Long' is ready for saving plots.
Starting analysis...
Loading combined data from /Users/gildagonzalez/Documents/Datos_Plots/Storms/Datos geomag/2020-2022.dat...
Data loaded successfully.
Loading wind data from /Users/gildagonzalez/Documents/Datos_Plots/SEA/MIGHTI/With_Long/Wind_Data_2020_2022_Long.dat...


/var/folders/42/1qzs7zpd7x9gc042ykb3byhm0000gn/T/ipykernel_1267/4276137501.py:85: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['UTC Time'] = pd.to_datetime(df['UTC Time'], errors='coerce', infer_datetime_format=True)


Wind data loaded successfully.
Columns in wind data: Index(['ICON_L22_Latitude', 'ICON_L22_Longitude', 'Meridional Wind (m/s)',
       'Zonal Wind (m/s)', 'Solar Local Time'],
      dtype='object')
Identified storm events with their minimum SYM-H values:
2020-04-20 12:00:00: SYM-H = -61.5
2020-04-20 13:00:00: SYM-H = -56.81666666666667
2020-07-25 00:00:00: SYM-H = -59.266666666666666
2020-07-25 01:00:00: SYM-H = -62.9
2020-08-31 00:00:00: SYM-H = -56.53333333333333
2020-08-31 01:00:00: SYM-H = -52.46666666666667
2020-09-25 22:00:00: SYM-H = -53.6
2020-09-25 23:00:00: SYM-H = -61.15
2020-09-26 00:00:00: SYM-H = -53.68333333333333
2020-09-26 01:00:00: SYM-H = -54.3
2020-09-26 02:00:00: SYM-H = -55.4
2020-09-26 03:00:00: SYM-H = -51.85
2020-09-27 21:00:00: SYM-H = -61.516666666666666
2020-09-27 22:00:00: SYM-H = -56.416666666666664
2020-09-27 23:00:00: SYM-H = -53.43333333333333
2020-09-28 00:00:00: SYM-H = -53.8
2020-09-28 01:00:00: SYM-H = -52.25
2020-09-28 04:00:00: SYM-H = -51.1833333